In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException

In [59]:
url = 'https://finance.naver.com/sise/etf.naver'

In [60]:
# 브라우저 종료 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option('detach', True)

driver = webdriver.Chrome(options=chrome_options)

In [61]:
driver.get(url)
driver.implicitly_wait(2)
soup = BeautifulSoup(driver.page_source, 'lxml')

In [62]:
etf_name = 'SOL 조선TOP3플러스'
start_date = '2024.08.01' # input('시작일을 입력하세요 (YYYY.MM.DD): ')
end_date = '2025.01.09' # input('종료일을 입력하세요 (YYYY.MM.DD): ')

In [63]:
# ETF 종목 페이지 URL 가져오기기
detail_url = 'https://finance.naver.com' + soup.select_one('#etfItemTable').find('a', string=etf_name).attrs['href']
detail_url

'https://finance.naver.com/item/main.naver?code=466920'

In [64]:
# ETF 종목 토론실 링크 주소 가져오기
discussion_bbs_url = detail_url.replace('main', 'board')
discussion_bbs_url

'https://finance.naver.com/item/board.naver?code=466920'

In [65]:
# 날짜 형식 변환
start_date = pd.to_datetime(start_date, format='%Y.%m.%d')
end_date = pd.to_datetime(end_date, format='%Y.%m.%d')

start_date, end_date

(Timestamp('2024-08-01 00:00:00'), Timestamp('2025-01-09 00:00:00'))

In [66]:
post_links = []         # 게시글 제목 url
post_date = []          # 게시글 작성일
post_title = []         # 게시글 제목
post_view_count = []    # 게시글 조회수
post_empathy = []       # 공감
post_dislike = []       # 비공감
post_contents = []      # 게시글 내용용

In [67]:
# 페이지 번호 초기화
page_number = 1
stop = False

In [68]:
while not stop:
  current_page_url = f"{discussion_bbs_url}&page={page_number}"
  driver.get(current_page_url)
  # 게시글 요소 가져오기
  trs = driver.find_elements(By.CSS_SELECTOR, '#content > div.section.inner_sub > table.type2 > tbody > tr')
  if len(trs) <= 4:  # 게시글이 없는 경우 종료
    print("더 이상 게시글이 없습니다.")
    stop = True
  # td 요소가 6개인 trs 필터링
  post_rows = [tr for tr in trs if len(tr.find_elements(By.TAG_NAME, 'td')) == 6]
  for tr in post_rows:
      cols = tr.find_elements(By.TAG_NAME, "td")

      # 2024-12-17 추가 : '클린봇이 이용자 보호를 위해 숨긴 게시글입니다.' 게시글의 경우 td 안에 a 태그가 없기때문에, 이 조건으로 거르기 전에는 에러가 발생했었음
      if len(cols[1].find_elements(By.TAG_NAME, "a")) > 0:                        # 2024-12-17 추가한 조건
        # 제목목
        title_ele = cols[1].find_element(By.TAG_NAME, "a")
        # 날짜 추출 (YYYY.MM.DD HH:MM)
        post_date_str = cols[0].text.strip()
        # 문자열을 날짜 타입으로 변경
        post_date_obj = pd.to_datetime(post_date_str, format='%Y.%m.%d %H:%M')

        # 날짜 필터링
        if start_date <= post_date_obj <= end_date:
          post_date.append(post_date_str)                         # 날짜
          post_title.append(title_ele.get_attribute('title'))     # 게시글 제목
          post_view_count.append(cols[3].text.strip())            # 조회수
          post_empathy.append(cols[4].text.strip())               # 공감
          post_dislike.append(cols[5].text.strip())               # 비공감
          post_links.append(title_ele.get_attribute('href'))      # 게시글 본문을 찾기위한 링크

      else:           # td가 정상적으로 6개 존재하지만 a태그가 없는 tr -> 클린봇이 차단한 게시글이라는 것을 파악함
        continue
  # 마지막 게시글의 작성날짜 확인
  last_post_date_str = post_rows[-1].find_elements(By.TAG_NAME, "td")[0].text.strip()
  last_post_date_obj = pd.to_datetime(last_post_date_str, format='%Y.%m.%d %H:%M')
  # 현재 페이지가 마지막 페이지 인지 확인
  try:
    # 마지막 페이지 에서는 맨끝 버튼이 없는 점을 이용
    driver.find_element(By.CSS_SELECTOR,
                        '#content > div.section.inner_sub > table.tbl_pagination > tbody > tr > td:nth-child(2) > table > tbody > tr > td.pgRR')
    # 마지막 페이지가 아닐 경우, 시작일 보다 현재 페이지의 마지막 글 작성 날짜가 미래 라면, 페이지 +1
    if start_date <= last_post_date_obj:
      page_number += 1
    else:
      stop = True
  except NoSuchElementException:
    # 마지막 페이지인 경우
    print("현재 페이지가 마지막 페이지 입니다.")
    stop = True
  except Exception as e:
    # 다른 예외가 발생한 경우
    print(f"예상치 못한 오류 발생: {e}")

In [69]:
# 게시글 본문 수집
for post_link in post_links:
  driver.get(post_link)
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#body")))
  post_soup = BeautifulSoup(driver.page_source, 'html.parser')

  # 본문 내용 정리
  body_content = post_soup.select_one("#body")
  if body_content:
    post_contents.append(body_content.text.strip().replace('\n', ' '))
  else:
    post_contents.append("본문 없음")

In [70]:
post_infos = {
    '날짜': post_date,
    '제목': post_title,
    '조회수': post_view_count,
    '공감': post_empathy,
    '비공감': post_dislike,
    '본문': post_contents
}

In [71]:
post_df = pd.DataFrame(post_infos)
print(post_df)

                  날짜                      제목   조회수  공감 비공감  \
0   2025.01.08 19:36          평단들  어떻게 대시나요?   291   4   2   
1   2025.01.08 14:44   1만주 16,500 부근 전량 매도..   423   3   2   
2   2025.01.08 09:06                냅두믄 천국간다   484   2   1   
3   2025.01.05 20:36           조선 산업 분석글입니다.  1273   2   4   
4   2025.01.02 14:02                올해 대장etf  2197   9   1   
..               ...                     ...   ...  ..  ..   
83  2024.08.05 16:58  12440매수 후 5분 기분 좋았습니다.  2062   2   0   
84  2024.08.05 13:48               본전에서 팔았다…   828   3   0   
85  2024.08.05 11:18                  미친거아녀?  1072   3   0   
86  2024.08.01 13:19                가는넘만 간다.  1329   1   3   
87  2024.08.01 11:36                이거밖에는...  1398  12   0   

                                                   본문  
0   평단 9171 수량 9200주  오늘 종가 수익률 +80.55%  1년만에 이렇게나...  
1   저번달 초중순 13,000 밑으로 한창 떨어질때 허겁지겁 주워담은거 오늘 한번에 털...  
2                                              헤으응...  
3       https://blog.naver.com/

In [72]:
post_df.to_csv(f'etf_discussion_posts_{etf_name}.csv', index=False, encoding='utf-8-sig')

In [73]:
# 드라이버 종료
driver.quit()

In [74]:
post_df.shape

(88, 6)